# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import yfinance as yf

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataprojectyf as dp


# Read and clean data

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [9]:
# a. fetch the data for the house representatives
house_raw = dp.fetch_data(mode="house", print_head = True)

# b. clean the data
house_clean = dp.clean_data(house_raw, print_head = True)

request successful


,date,ticker,amount,action,representative,party,description
0,2021-09-27,BP,"$1,001 - $15,000",purchase,Virginia Foxx,Republican,BP plc
1,2021-09-13,XOM,"$1,001 - $15,000",purchase,Virginia Foxx,Republican,Exxon Mobil Corporation
2,2021-09-10,ILPT,"$15,001 - $50,000",purchase,Virginia Foxx,Republican,Industrial Logistics Properties Trust - Common...
3,2021-09-28,PM,"$15,001 - $50,000",purchase,Virginia Foxx,Republican,Phillip Morris International Inc
4,2021-09-17,BLK,"$1,001 - $15,000",sale_partial,Alan S. Lowenthal,Democrat,BlackRock Inc
...,...,...,...,...,...,...,...
16916,2020-04-09,SWK,"$1,001 - $15,000",sale_partial,Ed Perlmutter,Democrat,"Stanley Black & Decker, Inc."
16917,2020-04-09,USB,"$1,001 - $15,000",sale_partial,Ed Perlmutter,Democrat,U.S. Bancorp
16918,2020-03-13,BMY,"$100,001 - $250,000",sale_full,Van Taylor,Republican,Bristol-Myers Squibb Company
16919,2020-03-13,LLY,"$500,001 - $1,000,000",sale_full,Van Taylor,Republican,Eli Lilly and Company


12 invalid dates dropped
1708 invalid tickers dropped
75 options trades dropped


,date,ticker,action,representative,party,description,min_amount,max_amount
0,2021-09-27,BP,purchase,Virginia Foxx,rep,BP plc,1001.0,15000.0
1,2021-09-13,XOM,purchase,Virginia Foxx,rep,Exxon Mobil Corporation,1001.0,15000.0
2,2021-09-10,ILPT,purchase,Virginia Foxx,rep,Industrial Logistics Properties Trust - Common...,15001.0,50000.0
3,2021-09-28,PM,purchase,Virginia Foxx,rep,Phillip Morris International Inc,15001.0,50000.0
4,2021-09-17,BLK,sale_partial,Alan S. Lowenthal,dem,BlackRock Inc,1001.0,15000.0
...,...,...,...,...,...,...,...,...
15121,2020-04-09,SWK,sale_partial,Ed Perlmutter,dem,"Stanley Black & Decker, Inc.",1001.0,15000.0
15122,2020-04-09,USB,sale_partial,Ed Perlmutter,dem,U.S. Bancorp,1001.0,15000.0
15123,2020-03-13,BMY,sale_full,Van Taylor,rep,Bristol-Myers Squibb Company,100001.0,250000.0
15124,2020-03-13,LLY,sale_full,Van Taylor,rep,Eli Lilly and Company,500001.0,1000000.0


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

# Analysis

# Conclusion